In [1]:
# Standard Library Imports
import pandas as pd
import re

# Third-party Library Imports# Standard Library Imports
import pandas as pd
import re
import datetime

# Third-party Library Imports
import requests
from bs4 import BeautifulSoup
import praw
import nltk
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import time
import itertools
from collections import defaultdict
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk.util import bigrams

# Custom Functions or Classes (if applicable)

import requests
from bs4 import BeautifulSoup
import praw
import nltk
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import time
import itertools
from itertools import combinations
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
import datetime
from nltk.stem import PorterStemmer
from collections import Counter
from nltk.util import bigrams
from sklearn.feature_extraction.text import CountVectorizer

# Custom Functions or Classes (if applicable)

In [2]:
# Set pandas display options to show the entire content of the "Post Text" column
pd.set_option('display.max_colwidth', None)

In [3]:
# Initialize the Reddit API client
redditscrapper = praw.Reddit(
    client_id='mTKAc7piwaoiD3fvkhY7qA',
    client_secret='GdT29i_cBYDTJwb0eExYEh6prVceGg',
    user_agent='(REDACTED NAME HERE)'
)

In [4]:
# List of subreddit names to scrape
subreddit_names = ["intermittentfasting", "AnorexiaNervosa"]

In [5]:
# Dictionary to store post data
posts_dict = {
    "Title": [],
    "Post Text": [],
    "ID": [],
    "Score": [],
    "Total Comments": [],
    "Post URL": [],
    "Subreddit": [],
    "Post Type": [],  # Add a column for post type (new, hot, top, rising)
    "Time uploaded": []
}

In [6]:
# Set to keep track of collected post IDs
collected_post_ids = set()

In [7]:
# Iterate through the subreddit names to test accessibility
for subreddit_name in subreddit_names:
    reddit_url = f"https://www.reddit.com/r/{subreddit_name}"
    response = requests.get(reddit_url)

    if response.status_code == 200:
        print(f"Success! The subreddit at {reddit_url} is accessible.")
    else:
        print(f"Error! The subreddit at {reddit_url} returned a status code of {response.status_code}.")

Success! The subreddit at https://www.reddit.com/r/intermittentfasting is accessible.
Success! The subreddit at https://www.reddit.com/r/AnorexiaNervosa is accessible.


In [8]:
# Set to keep track of collected post IDs
collected_post_ids = set()

# Iterate through the subreddit names and post types to fetch and collect posts
for subreddit_name in subreddit_names:
    subreddit = redditscrapper.subreddit(subreddit_name)
    post_types = ["hot", "new", "top", "rising"]  # You can add more if needed
    
    for post_type in post_types:
        if post_type == "new":
            posts = subreddit.new(limit=1000)  # Fetch new posts
        elif post_type == "hot":
            posts = subreddit.hot(limit=1000)  # Fetch hot posts
        elif post_type == "top":
            posts = subreddit.top(limit=1000)  # Fetch top posts
        elif post_type == "rising":
            posts = subreddit.rising(limit=1000)  # Fetch rising posts
        
        for post in posts:
            if post.id not in collected_post_ids:
                collected_post_ids.add(post.id)
                posts_dict["Title"].append(post.title)
                posts_dict["Post Text"].append(post.selftext)
                posts_dict["ID"].append(post.id)
                posts_dict["Score"].append(post.score)
                posts_dict["Total Comments"].append(post.num_comments)
                posts_dict["Post URL"].append(post.url)
                posts_dict["Subreddit"].append(subreddit_name)
                posts_dict["Post Type"].append(post_type)
                posts_dict["Time uploaded"].append(post.created_utc)

# Create a DataFrame from the collected data
all_posts = pd.DataFrame(posts_dict)

# Save the data to a CSV file
all_posts.to_csv("reddit_posts (250923 1710).csv", index=False)

# Print a summary of the collected data
print(f"Total number of posts collected: {len(all_posts)}")
all_posts.head()

Total number of posts collected: 3961


,Title,Post Text,ID,Score,Total Comments,Post URL,Subreddit,Post Type,Time uploaded
0,Daily Fasting Check-in!,"* **Type** of fast (water, juice, smoking, etc.)\n* **Context** of fast (start, end, day x of y, etc.)\n* **Length** of fast (8 hours, 3 days, etc.)\n* **Why?** What you hope to accomplish with your fast\n* **Notes** How is it going so far? Any concerns? Insights to share?\n\nBe sure to check back often as comments get posted throughout the day. Sort comments by ""new"" to be sure the newer comments get some love as well.",16rk06v,3,0,https://www.reddit.com/r/intermittentfasting/comments/16rk06v/daily_fasting_checkin/,intermittentfasting,hot,1.695622e+09
1,6 months of dedication | 280lbs vs 205lbs 6'4,,16rari2,2678,82,https://www.reddit.com/gallery/16rari2,intermittentfasting,hot,1.695594e+09
2,I did my first pullup,"After losing 15kg this year (33 pounds), I can finally pull my own body weight for the first time in my 33 years of existence.\nIt's just two pull ups but I am extremely happy as I've never succeeded in doing a single pull up before! \nI owe it to this sub, to everything I learned about food, nutrition, IF, mindset here.\nStay hard!!!",16rn8qu,80,11,https://v.redd.it/ajlxmiz2edqb1,intermittentfasting,hot,1.695634e+09
3,As of this morning I have officially lost 1 stone,,16rc016,321,18,https://i.redd.it/nchnam1qdaqb1.jpg,intermittentfasting,hot,1.695597e+09
4,"Yeah, I'm proud of myself for this. 30lbs gone since July!","First picture is from today, second picture from July!",16rl3su,76,6,https://www.reddit.com/gallery/16rl3su,intermittentfasting,hot,1.695626e+09


[Click here to see the notebook used to clean the scrapped data](Project 3 Cleaning (caa 250923 2057)).ipynb)